In [ ]:
import os, glob, json, warnings
import pandas as pd
import importlib.util
from coffea import processor
from coffea.nanoevents import NanoAODSchema

In [ ]:
### Directory Setup ###
outputPath = "data/results/makeDF/2018/mc/"
os.makedirs(outputPath, exist_ok=True)

In [ ]:
### TaskVine Setup ###
manager_name = os.environ.get("VINE_MANAGER_NAME")
print(manager_name)
ports_str = os.environ.get("VINE_MANAGER_PORTS", "9123, 9150")
ports = [int(p.strip()) for p in ports_str.split(",")]

In [ ]:
if len(ports) == 1:
    ports = ports[0]
else:
    ports = [int(p) for p in ports]

In [ ]:
print(f"Manager Ports: {ports}")

In [ ]:
executor_args = {
    'desc'             : f'Processing makeDF',
    'manager_name'     : manager_name,
    'port'             : ports,
    'extra_input_files': glob.glob(f"utils/*"),
    'retries'          : 5,
    'fast_terminate_workers': 0
}
executor = processor.TaskVineExecutor(**executor_args)

In [ ]:
print(f"nCores  : {int(os.cpu_count())}\nnWorkers: 10")

In [ ]:
samples={"ETau":[f for f in os.listdir("data/samples/test/ETauData/")],
                "DYJets":[f for f in os.listdir("data/samples/test/DYData/")]}

In [ ]:
# module
processorpath = 'processors/makeDF.py' 
spec   = importlib.util.spec_from_file_location('my_processor', processorpath)
module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(module)

In [ ]:
# processor
warnings.filterwarnings("ignore", message=r"Missing cross-reference.*", module="coffea.nanoevents.schemas.nanoaod")
processor_instance = module.my_processor(year="2018", type="mc")
run = processor.Runner(
    executor      = executor,
    schema        = NanoAODSchema,
    maxchunks     = None,
    chunksize     = 20000,
    xrootdtimeout = 900,
    skipbadfiles  = False
)

In [ ]:
### Running & Saving ###
result = run(samples, treename="Events", processor_instance=processor_instance)
for sampleName in samples.keys():
    dicts = {}
    for key, value in result.items():
        if sampleName==key[1]: dicts.update({key[0]:value})
    parqResult = pd.DataFrame([{key[0]:value for key,value in result.items() if sampleName in key[1]}])
    parqResult.to_parquet(f"{outputPath}/{sampleName}.parq")
    print(f"Results saved in {outputPath}/{sampleName}.parq")